## About this notebook

In this notebook, I followed the tutorial notebook to build a machine learning model to classify 4 types of cassaval leaf disease and 1 healthy type based on their images.

I am going to train the model on a Tensor Processing Unit (TPU).


# Setups #

In [ ]:
## Basic packages
import math, re, os, random, warnings, glob, cv2, gc
warnings.simplefilter(action = 'ignore', category = FutureWarning)

## For data handling
import pandas as pd
import numpy as np
from functools import partial
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## Tensorflow packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import tensorflow.keras.backend as K

print("Tensorflow version " + tf.__version__)

In [ ]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

SEED = 414
seed_everything(SEED)

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

REPLICAS = strategy.num_replicas_in_sync

print("REPLICAS: ", REPLICAS)

# Loading Data #

In [ ]:
#from kaggle_datasets import KaggleDatasets

#GCS_DS_PATH = KaggleDatasets().get_gcs_path('cassava-leaf-disease-classification')
GCS_DS_PATH = '../input/cassava-leaf-disease-classification'
print(GCS_DS_PATH)

In [ ]:
BATCH_SIZE = 16 * REPLICAS
WARMUP_EPOCHS = 3
WARMUP_LEARNING_RATE = 1e-4 * REPLICAS
EPOCHS = 20
LEARNING_RATE = 5e-5 * REPLICAS
ES_PATIENCE = 5

CHANNELS = 3
N_CLASSES = 5
DIM = 512
HEIGHT = 512
WIDTH = 512
CLASSES = ['0', '1', '2', '3', '4']

#model_path = f'model_efn.h5'
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
ROT_ = 180.0
SHR_ = 2.0
HZOOM_ = 8.0
WZOOM_ = 8.0
HSHIFT_ = 8.0
WSHIFT_ = 8.0

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

In [ ]:
def transform(image, DIM=512):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    XDIM = DIM%2 #fix for size 331
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(DIM//2, -DIM//2,-1), DIM)
    y   = tf.tile(tf.range(-DIM//2, DIM//2), [DIM])
    z   = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM, DIM,3])

In [ ]:
def read_labeled_tfrecord(example):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'target'                       : tf.io.FixedLenFeature([], tf.int64)
    }           
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['target']


def read_unlabeled_tfrecord(example, return_image_name):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        'image_name'                   : tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['image_name'] if return_image_name else 0

 
def prepare_image(img, augment=True, dim=512):    
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    
    if augment:
        img = transform(img,DIM=dim)
        img = tf.image.random_flip_left_right(img)
        #img = tf.image.random_hue(img, 0.01)
        #img = tf.image.random_saturation(img, 0.7, 1.3)
        #img = tf.image.random_contrast(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.1)
                      
    img = tf.reshape(img, [dim,dim, 3])
            
    return img

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) 
         for filename in filenames]
    return np.sum(n)

In [ ]:
def get_dataset(files, augment = False, shuffle = False, repeat = False, 
                labeled=True, return_image_names=True, batch_size=BATCH_SIZE, dim=512):
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = ds.cache()
    
    if repeat:
        ds = ds.repeat()
    
    if shuffle: 
        ds = ds.shuffle(1024*8)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
        
    if labeled: 
        ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)
    else:
        ds = ds.map(lambda example: read_unlabeled_tfrecord(example, return_image_names), 
                    num_parallel_calls=AUTO)      
    
    ds = ds.map(lambda img, imgname_or_label: (prepare_image(img, augment=augment, dim=dim), 
                                               imgname_or_label), 
                num_parallel_calls=AUTO)
    
    ds = ds.batch(batch_size * REPLICAS)
    ds = ds.prefetch(AUTO)
    return ds

### Train test split

In [ ]:
TEST_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH + '/test_tfrecords/*.tfrec')
#TEST_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH + '/train_tfrecords/ld_train08-1338.tfrec')

In [ ]:
print(TEST_FILENAMES)

In [ ]:
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print('Dataset: {} unlabeled test images'.format(NUM_TEST_IMAGES))

### Define Model

In [ ]:
import sys
sys.path.append('/kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle')
! pip install /kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle

In [ ]:
# import DenseNet201, Xception, InceptionV3, and InceptionResNetV2
import efficientnet.keras as efn
#from tensorflow.keras.applications import DenseNet201, Xception

In [ ]:
def create_model_efnB6():
    base_model = efn.EfficientNetB6(weights=None,
                                          include_top=False,
                                          input_shape=[HEIGHT, WIDTH, 3])
    #base_model.trainable = False # Freeze layers
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])
    
    return model

In [ ]:
with strategy.scope():
     model_efnB6 = create_model_efnB6()

In [ ]:
TTA = 1
print('Predicting Test with TTA...')
test_ds = get_dataset(TEST_FILENAMES,labeled=False,return_image_names=False,augment=False,
                      repeat=False,shuffle=False)
test_ct = count_data_items(TEST_FILENAMES); 
STEPS = TTA * test_ct/BATCH_SIZE/REPLICAS
if STEPS < 1:
    STEPS = 1

test_df = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
probabilities = np.zeros((test_df.shape[0],5))

for f in range(5):
    model_efnB6.load_weights("../input/cassava-leaf-disease-classification-kfold/model_efnB6_train_fold_%i.h5"%f, by_name=True)
    #prob = model_efnB4.predict(test_ds,steps=STEPS,verbose=2)[:TTA*test_ct,]
    prob = model_efnB6.predict(test_ds, verbose=2)
    probabilities += prob/5

#model_efnB6.load_weights("../input/cassava-leaf-disease-classification-kfold/model_efnB6_train_fold_0.h5", by_name=True)
#probabilities = model_efnB6.predict(test_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)
print(probabilities)

In [ ]:
print('Generating submission.csv file...')
ds = get_dataset(TEST_FILENAMES,labeled=False,return_image_names=True,augment=False,
                      repeat=False,shuffle=False)
# Get image ids from test set and convert to unicode
test_ids = np.array([img_name.numpy().decode("utf-8") 
                        for img, img_name in iter(ds.unbatch())])

# Write the submission file
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([test_ids, predictions]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='image_id,label',
    comments='',
)

# Look at the first few predictions
!head submission.csv